In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
root_dir = '/content/drive/My Drive/Colab Notebooks/NLP'
os.chdir(root_dir)

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_csv('train_LSTM.csv', encoding='latin1', on_bad_lines='warn')

<ipython-input-2-5567a11a1c59>:3: ParserWarning: Skipping line 49: expected 5 fields, saw 7
Skipping line 50: expected 5 fields, saw 6
Skipping line 51: expected 5 fields, saw 6
Skipping line 54: expected 5 fields, saw 6
Skipping line 257: expected 5 fields, saw 95
Skipping line 258: expected 5 fields, saw 20
Skipping line 292: expected 5 fields, saw 25
Skipping line 319: expected 5 fields, saw 49
Skipping line 359: expected 5 fields, saw 47
Skipping line 399: expected 5 fields, saw 8
Skipping line 404: expected 5 fields, saw 6
Skipping line 450: expected 5 fields, saw 6
Skipping line 486: expected 5 fields, saw 363
Skipping line 527: expected 5 fields, saw 56
Skipping line 534: expected 5 fields, saw 7
Skipping line 551: expected 5 fields, saw 7
Skipping line 552: expected 5 fields, saw 10
Skipping line 592: expected 5 fields, saw 7
Skipping line 604: expected 5 fields, saw 7
Skipping line 612: expected 5 fields, saw 6
Skipping line 615: expected 5 fields, saw 6
Skipping line 617: exp

In [3]:
X = df.drop("label", axis=1)

In [4]:
y = df['label']

In [14]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [6]:
# Dataset Preprocessing
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [51]:
messages = X.copy()
messages.reset_index(inplace = True)

In [53]:
 ### Dataset Preprocessing
 ps = PorterStemmer()
 corpus = []
 for i in range(0,len(messages)):
  sent = re.sub('[^a-zA-Z]', ' ' ,  messages['title'][i])
  sent = sent.lower()
  sent = sent.split()

  sent = [ps.stem(word) for word in sent if word not in nltk.corpus.stopwords.words('english')]
  sent = ' '.join(sent)
  corpus.append(sent)


print(corpus)

TypeError: expected string or bytes-like object, got 'float'

In [21]:
# Embedding
# one hot encoding

voc_size = 5000
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2684, 3678, 3128, 4894, 3180, 561, 1704, 838, 4197, 3424],
 [4993, 4152, 3622, 1409, 2854, 1191, 1305],
 [797, 1109, 1530, 1186],
 [3042, 2515, 1398, 4978, 2637, 93],
 [1159, 2854, 3767, 4341, 3361, 4966, 2854, 755, 3427, 2065],
 [1137,
  1473,
  1793,
  4338,
  4741,
  4711,
  4596,
  2847,
  33,
  3739,
  815,
  4270,
  4837,
  2205,
  1305],
 [1146, 1146, 1565, 1723, 499, 4764, 4353, 4152, 1887, 2996, 3415, 3870],
 [2696, 3537, 4528, 1462, 4476, 1030, 2626, 1830, 1048, 180, 845],
 [493, 475, 2457, 366, 4711, 751, 499, 4063, 354, 780, 1048, 180, 845],
 [1318, 4294, 4407, 2505, 891, 1939, 4711, 2979, 1048, 180, 845],
 [1542, 116, 1075, 94, 4053, 1514, 4809, 4132, 4711, 2677],
 [4197, 4469, 1397, 1958, 1724, 63, 4210, 3479],
 [3631, 694, 177, 602, 2886, 2225, 4549, 1013, 2497, 1627, 2648],
 [4978, 3011, 3180, 1514, 4711, 891],
 [3861, 3894, 4068, 732, 1431, 4348, 2837, 4342, 4934],
 [2684, 3331, 4456, 4161, 3016, 276, 4407, 1048, 180, 845],
 [2248, 4147, 4195, 4880, 1768, 1048, 180, 

In [22]:
# pad_sequences
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=30)
print(embedded_docs)

[[2684 3678 3128 ...    0    0    0]
 [4993 4152 3622 ...    0    0    0]
 [ 797 1109 1530 ...    0    0    0]
 ...
 [2316 3904 4404 ...    0    0    0]
 [ 997  891  805 ...    0    0    0]
 [ 611  347  219 ...    0    0    0]]


In [23]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=32)


In [47]:
# create model
in_dim = voc_size
out_dim = 40
max_len = 30
model = Sequential()
model.add(Embedding(in_dim, out_dim, input_length = max_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.build(input_shape=(None, 50))

model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 50, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model.compile(metrics=['acc'], optimizer = 'adam', loss='binary_crossentropy')

In [ ]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))